# Task 4

Conversion of a measurement of atmospheric liquid water path from binary format into netcdf format

## 1. Load modules

In [1]:
import struct
import xarray as xr
import datetime

## 2. Data preparation
Read the binary file and do the conversion according to the Manual_Profilers

In [2]:
# Initial the empty lists for the variables T, RF, LWP, and LWPANG
T      = []
RF     = []
LWP    = []
LWPAng = []

# Open the binary file
with open('Data/04-Conversion_of_binary_to_netcdf/21060900.LWP', mode='rb') as file: # b is important -> binary
    # According to Manual_Profilers, read following attributes and variables
    # Read 6 headings firstly, according to the bytes of every variable
    LWPCode, N, LWPMin, LWPMax, LWPTimeRef, LWPRetrieval = struct.unpack('<IIffII', file.read(4 * 6))
    # Based on the number of recorded samples, loop the data
    for i in range(N):
        T.append(struct.unpack('<I', file.read(4)))      # int
        RF.append(struct.unpack('<b', file.read(1)))     # char
        LWP.append(struct.unpack('<f', file.read(4)))    # float
        LWPAng.append(struct.unpack('<f', file.read(4))) # float

# Check if the reading method is right
if (LWPCode==934501978):
    print('reading method is right')

reading method is right


### Convert the time of seconds to UTC time

In [3]:
T_converted = []
# 2001-01-01 00:00:00 is the reference time
reference_time = datetime.datetime(year=2001, month=1, day=1, hour=0, minute=0, second=0)
for itime in T:
    time_delta = datetime.timedelta(seconds = itime[0])
    sample_time = reference_time + time_delta
    T_converted.append(sample_time)

## 3. Create the NetCDF file

In [4]:
ds = xr.Dataset(
    # Data variables
    data_vars=dict(
        T   = (["time"], np.hstack(T)),
        RF  = (["time"], np.hstack(RF)),
        LWP = (["time"], np.hstack(RF)),
        LWPAng = (["time"], np.hstack(LWPAng)),
    ),
    # Data coordinates
    coords=dict(
        sample = (["sample"], np.arange(0,N,1)),
        time= np.hstack(T_converted),
        reference_time = reference_time
    ),
    # Global attributes
    attrs=dict(
        LWP_File_Code = LWPCode,
        Number_of_recorded_samples = N,
        Minimum_of_recorded_LWP_values = LWPMin,
        Maximum_of_recorded_LWP_values = LWPMax,
        Time_reference = str(LWPTimeRef)+' UTC',
        LWPRetrieval   = str(LWPRetrieval)+' 0: lin. Reg., 1 : quad. Reg., 2: Neur. Net.'
    )
)

# Variable attributes
ds.T.attrs=dict(Description='Time of sample (# of sec. since 1.1.2001)')
ds.RF.attrs=dict(Description='Rainflag of sample 1 (0: no rain, 1: rain)')
ds.LWP.attrs=dict(Description='LWP sample 1 [g/m^2]')
ds.LWPAng.attrs=dict(Description='LWP angle 1 [DEG]')

# Show the nc file
ds

<xarray.Dataset>
Dimensions:         (sample: 2193, time: 2193)
Coordinates:
  * sample          (sample) int64 0 1 2 3 4 5 ... 2187 2188 2189 2190 2191 2192
  * time            (time) datetime64[ns] 2021-06-09T00:02:25 ... 2021-06-09T...
    reference_time  datetime64[ns] 2001-01-01
Data variables:
    T               (time) int64 644889745 644889747 ... 644893199 644893200
    RF              (time) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    LWP             (time) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    LWPAng          (time) float64 2.401e+05 2.401e+05 ... 2.401e+05 2.401e+05
Attributes:
    LWP_File_Code:                   934501978
    Number_of_recorded_samples:      2193
    Minimum_of_recorded_LWP_values:  -5.0
    Maximum_of_recorded_LWP_values:  6.667603969573975
    Time_reference:                  1 UTC
    LWPRetrieval:                    2 0: lin. Reg., 1 : quad. Reg., 2: Neur....

## 4. Save the NetCDF file

In [5]:
ds.to_netcdf(path='Data/04-Conversion_of_binary_to_netcdf/LWP_2021060900.nc')